In [1]:
import pandas as pd

df = pd.ExcelFile('./NIHMS40463-supplement-6.xls')
df = pd.read_excel('./NIHMS40463-supplement-6.xls', header = None)

In [2]:
dnds = df[[0, 4]]
dnds = dnds[1:]
dnds.columns = ['gene_name', 'm0_dnds']
dnds.head()

,gene_name,m0_dnds
1,SUPT16H,0.04344
2,RBBP4,0.06444
3,MOBKL3,0.18094
4,ATP11A,0.08728
5,ARHGAP5,0.08563


In [3]:
for_color = pd.read_csv('./R_agr_united.csv')
for_color.head()

,symbol,res
0,A1CF,3.565082
1,A2M,2.932616
2,A2ML1,9.173608
3,A4GNT,8.270570
4,AACS,7.370222


In [6]:

names = ['Akt_mTOR_EGF_Pathway', 'Akt_Signaling_Pathway', 'EGF_Pathway', 'Hedgehog_Signaling_in_Mammals_no_Tubulin_Pathway',
         'mTOR_Pathway', 'Notch_Signaling_Pathway', 'WNT_Pathway', 'WNT-Notch-Hedgehog_Pathway_2', 'reactome_PI_3K_cascade_Main_Pathway']

for name in names:
    file = './'+ name + '.xlsx'
    df_cur = pd.ExcelFile(file)
    genes = pd.read_excel(df_cur, 'genes', header = None)
    edges = pd.read_excel(df_cur, 'edges', header = None)
    nodes = pd.read_excel(df_cur, 'nodes', header = None)
    nodes.columns = ['node', 'role', 'composition']
    size = []
    color = []
    comp = nodes['composition'].tolist()
    for c in comp:
        c = str(c).split(',')
        s = 0
        col = 0
        for i in range(len(c)):
            for j in range(1,len(dnds)):
                if dnds.at[j, 'gene_name'] == c[i]:
                    s += float(dnds.at[j, 'm0_dnds'])
        for j in range(len(for_color)):
                if for_color.at[j, 'symbol'] == c[i]:
                    col += float(for_color.at[j, 'res'])
        size.append(s/len(c) + 0.3)
        color.append(col)
    nodes['size'] = size
    nodes['color'] = color
    filename = './res_with_size_and color/' + name + '_final' +'.xlsx'
    names_sheets = {'genes': genes, 'edges': edges, 'nodes': nodes}

    writer = pd.ExcelWriter(filename, engine='xlsxwriter')

    for sheet_name in names_sheets.keys():
        names_sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, header=True, index=False)
    writer.save()

## spearman

In [11]:
from scipy import stats
from scipy.stats import kendalltau, pearsonr, spearmanr
    
def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]



names = ['Akt_mTOR_EGF_Pathway', 'Akt_Signaling_Pathway', 'EGF_Pathway', 'Hedgehog_Signaling_in_Mammals_no_Tubulin_Pathway',
         'mTOR_Pathway', 'Notch_Signaling_Pathway', 'WNT_Pathway', 'WNT-Notch-Hedgehog_Pathway_2', 'reactome_PI_3K_cascade_Main_Pathway']
for name in names:
    file = './res_with_size_and color/' + name + '_final' +'.xlsx'
    df_cur = pd.ExcelFile(file)
    nodes = pd.read_excel(df_cur, 'nodes')
    nodes = nodes[(nodes.size != 0.3) & (nodes.color != 0)]
    #print(nodes.head())
    for_sp = nodes[['size', 'color']]
    size_sp = for_sp['size'].to_list()
    color_sp = for_sp['color'].to_list()
    res = for_sp.corr('spearman')
    res_p = for_sp.corr(method=spearmanr_pval)
    print(name, "\n")
    print('spearman via pandas')
    print('coef = ', str(res.at['size', 'color']))
    print('p-value = ', str(res_p.at['size', 'color']), "\n")
    print('spearman via scipy.stats')
    print(stats.spearmanr(size_sp, color_sp), "\n")

Akt_mTOR_EGF_Pathway 

spearman via pandas
coef =  0.06990430915655699
p-value =  0.43483281002455765 

spearman via scipy.stats
SpearmanrResult(correlation=0.069904309156557, pvalue=0.43483281002455765) 

Akt_Signaling_Pathway 

spearman via pandas
coef =  0.13050705114500671
p-value =  0.2815457274413621 

spearman via scipy.stats
SpearmanrResult(correlation=0.13050705114500671, pvalue=0.2815457274413621) 

EGF_Pathway 

spearman via pandas
coef =  -0.1367478112991747
p-value =  0.4064738549769995 

spearman via scipy.stats
SpearmanrResult(correlation=-0.13674781129917474, pvalue=0.4064738549769995) 

Hedgehog_Signaling_in_Mammals_no_Tubulin_Pathway 

spearman via pandas
coef =  -0.10714285714285714
p-value =  0.8191508555699912 

spearman via scipy.stats
SpearmanrResult(correlation=-0.10714285714285716, pvalue=0.8191508555699912) 

mTOR_Pathway 

spearman via pandas
coef =  0.14941855945104313
p-value =  0.32163943567592357 

spearman via scipy.stats
SpearmanrResult(correlation=0.14